In [1]:
# file_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overconsistency_gpt4o_eval.txt"

# with open(file_path, "r") as file:
#     lines = file.readlines()

# #format the output, remove the \n at the end of each line
# new_lines = [x for x in lines if x != "\n"]

# s = {}

# for index in range(0,180):
#     add = 0
#     output = ""
#     while(add<6):
#         output += new_lines[index*6+add]
#         add += 1
#     s[str(index)] = output

# import json

# file_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overall_consistency_gpt4eval_results.json"

# with open(file_path, "r") as file:
#     data = json.load(file)
# for key in data.keys():
#     data[key] = data[key].replace('\n\n','\n')
# ss = s|data
# print(len(s.keys()))
# # 使用 json 保存字典
# with open(r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overall_consistency_gpt4eval_results.json", "w") as f:
#     json.dump(ss, f,indent=4)

In [2]:
#把gpt4o_eval的结果加入到annotation中
# import json
# import os

# models = ['gen2','lavie','pika','show1','videocrafter2']

# file_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\overall_consistency_gpt4eval_results.json"
# with open(file_path, "r") as file:
#     data = json.load(file)

# video_path = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\overall_consistency.json"
# with open(video_path, "r") as file:
#     annotations = json.load(file)


# for index,key in enumerate(data.keys()):
#     for model in models:       
#         score = int(data[key][model].split(' ')[2][0])
#         annotations[index]['gpt4o_eval'][model] = score
# w_json = r"D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\overall_consistency_annotation.json"
# with open(w_json, "w") as f:
#     json.dump(annotations, f,indent=4)

In [3]:
# 把人类标注数据写入json文件 来源anno_res_anlysis.ipynb
# import json


# import pandas as pd
# anno_1 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\hanhui_oc.xlsx'
# anno_2 = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\humananno_res\lsy_oc.xlsx'

# anno_1oc =pd.read_excel(anno_1, usecols=['Model','overall_consistency'])
# anno_2oc =pd.read_excel(anno_2, usecols=['Model','overall_consistency'])
# anno_1oc = anno_1oc.astype(object).where(pd.notnull(anno_1oc), None)
# anno_2oc = anno_2oc.astype(object).where(pd.notnull(anno_2oc), None)

# jsonpath = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\overall_consistency_annotation.json'
# models = ['gen2','lavie','pika','show1','videocrafter2']
# with open(jsonpath, 'r') as f:
#     oc = json.load(f)
# for i in range(0,len(oc)):
#     startindex = 5*i
#     endindex = 5*(i+1)
#     for idnex in range(startindex,endindex):
#         model = anno_1oc['Model'][idnex].lower()
#         if anno_1oc['overall_consistency'][idnex]!=None and anno_2oc['overall_consistency'][idnex]!=None:
#             score1 = int(anno_1oc['overall_consistency'][idnex])
#             score2 = int(anno_2oc['overall_consistency'][idnex])
#             oc[i]['human_anno'][model] = [score1,score2]
# with open(jsonpath, 'w') as f:
#     json.dump(oc, f, indent=4)

In [4]:
#把GPT4o的结果写入excel文件
import json
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Border, Side

models = ['gen2','videocrafter2','pika','show1','lavie']
jsonpath = 'D:\Astudying\VideoEval\HighlyHumanLikeVEB\Human_anno\color.json'
with open(jsonpath, 'r') as f:
    oc = json.load(f)

df = pd.DataFrame(columns=['Prompt','Group','Model','color score'])
for index in range(0,len(oc)):
    prompt = oc[index]['prompt_en']
    group = index%3
    score = []
    for model in models:
        score.append(oc[index]['gpt4o_eval'][model])
    new_row = pd.DataFrame({'Prompt': [prompt]*5, 'Group': [group]*5, 'Model': models, 'color score': score})
    df = pd.concat([df, new_row], ignore_index=True)

df.to_excel('D:\Astudying\VideoEval\HighlyHumanLikeVEB\GPT4o_eval_results\color_gpt4o_eval.xlsx',index=False)


output_path = 'D:\\Astudying\\VideoEval\\HighlyHumanLikeVEB\\GPT4o_eval_results\\color_gpt4o_eval.xlsx'

# 调整列宽
wb = load_workbook(output_path)
ws = wb.active

for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # 获取列字母
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# 添加分割线
thin_border = Border(bottom=Side(style='thin'))
for row in range(6, ws.max_row + 1, 5):  # 从第6行开始，每5行添加一次
    for col in range(1, ws.max_column + 1):
        cell = ws.cell(row=row, column=col)
        cell.border = thin_border

wb.save(output_path)
